In [2]:
import redis
import os
import numpy as np
import pandas as pd
import pickle

r = redis.Redis(host='localhost', port=6379)

input_url = "bosch_cnc"
metadata_url = os.path.join(
    input_url,
    "metadata.csv"
)
metadata_df = pd.read_csv(metadata_url)
metadata_df.head()

,filename,time_frame,machine_id,op_id,tool_id,sensor_id,sampling_f,max,dtype,status
0,0.csv,1,0,3,0,0,2000,8192,<i2,bad
1,1.csv,2,0,3,0,0,2000,8192,<i2,good
2,2.csv,1,0,3,0,0,2000,8192,<i2,good
3,3.csv,0,0,3,0,0,2000,8192,<i2,good
4,4.csv,2,0,3,0,0,2000,8192,<i2,good


In [ ]:
DATA_COLUMNS = {
    "x": np.float32,
    "y": np.float32,
    "z": np.float32,
}
filenames = os.listdir(input_url)
filenames.remove('metadata.csv')
filenames.remove('.DS_Store')

for filename in filenames:
    file_path = os.path.join(input_url, filename)
    data_df = pd.read_csv(
        file_path, 
        names=DATA_COLUMNS.keys(),
        dtype=DATA_COLUMNS,
    )
    
    
    x = pickle.dumps(data_df['x'])
    y = pickle.dumps(data_df['y'])
    z = pickle.dumps(data_df['z'])
    mapping = {
        'x': x,
        'y': y,
        'z': z
    }
    signal_name = filename.split('.')[0]
    r.hset(name='signal:'+signal_name, mapping=mapping)
    


In [ ]:
FILES = []
for file in r.keys(pattern='signal:*'):
    FILES.append(int(file.decode('utf-8').split(':')[-1]))
FILES.sort()
print(FILES)

In [17]:
filename = 0
x_axis = pickle.loads(r.hget(name="signal:"+str(filename), key='x'))
y_axis = pickle.loads(r.hget(name="signal:"+str(filename), key='y'))
z_axis = pickle.loads(r.hget(name="signal:"+str(filename), key='z'))

signal_1 = {
    'x':x_axis,
    'y':y_axis
}
print(signal_1)

{'x': 0         1032.0
1         1011.0
2         1075.0
3         1056.0
4         1138.0
           ...  
139648      95.0
139649      66.0
139650     160.0
139651     -23.0
139652      27.0
Name: x, Length: 139653, dtype: float32, 'y': 0        -368.0
1         115.0
2         316.0
3         357.0
4         388.0
          ...  
139648   -160.0
139649   -298.0
139650   -275.0
139651    -27.0
139652    195.0
Name: y, Length: 139653, dtype: float32}
